In [1]:
#Dependencies and Setup

import pandas as pd
import numpy as np
from statistics import mean
#import statistics

fantasy_df = pd.read_csv('Resources/purchase_data.csv')
#rename to sales_data

# Observations

Heroes of Pymoli is a game dominated by men as 84% of players are male.
Majority of players are in their early twenties.
Majority of the most popular items are among some of the most expensive items.

# Player Count

In [2]:
player_count = pd.DataFrame([{'Total Players': 0}])
num_of_players = len(fantasy_df['SN'].unique())
player_count['Total Players'] = num_of_players
player_count

,Total Players
0,576


# Purchasing Analysis (Total)

In [3]:
purchase_analysis = pd.DataFrame(np.array([[0,0,0,0]]), columns=['Number of Unique Items', 'Average Price', 
                                          'Number of Purchases', 'Total Revenue'])
inventory_df = pd.DataFrame(fantasy_df[['Item ID','Item Name', 'Price']].drop_duplicates(['Item ID']))
inventory_df = inventory_df.set_index(['Item ID', 'Item Name'])
purchase_analysis['Number of Unique Items'] = len(inventory_df)
purchase_analysis['Average Price'] = mean(list(inventory_df['Price']))
purchase_analysis['Average Price'] = purchase_analysis['Average Price'].map("${:,.2f}".format)
purchase_analysis['Number of Purchases'] = len(fantasy_df['Item ID'])
purchase_analysis['Total Revenue'] = fantasy_df['Price'].sum()
purchase_analysis['Total Revenue'] = purchase_analysis['Total Revenue'].map("${:,.2f}".format)
purchase_analysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.04,780,"$2,379.77"


# Gender Demographics

In [4]:
players_df = fantasy_df.drop_duplicates(['SN'])
gender_demo = pd.DataFrame(players_df['Gender'].value_counts())
gender_demo = gender_demo.rename(columns={'Gender': 'Total Count'})
gender_demo['Percentage of Players (%)'] = round((gender_demo['Total Count']/num_of_players)*100,2)
gender_demo

,Total Count,Percentage of Players (%)
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


# Purchasing Analysis (Gender)

In [5]:
purchase_analysis = fantasy_df.groupby('Gender').agg({'Item ID': 'count', 'Price': ['sum', lambda x: round(x.unique().mean(),2)]})
purchase_analysis.columns = purchase_analysis.columns.droplevel(level=1)

#average purchase by person by gender
purchase_analysis2 = fantasy_df.groupby(['Gender', 'SN']).agg({'Item ID': 'count', 'Price': 'sum'})
purchase_analysis2 = purchase_analysis2.groupby('Gender').agg(
    Avg_Purchase_Tot = pd.NamedAgg(column = 'Price', aggfunc= lambda x: round(mean(x),2))
)

#final summary
purchase_analysis_gender = purchase_analysis.join(purchase_analysis2)

#rename columns
purchase_analysis_gender.columns = ['Number of Purchases', 'Total Revenue', 'Average Price', 'Avg Total Purchase Per Person']

#format columns
purchase_analysis_gender['Total Revenue'] = purchase_analysis_gender['Total Revenue'].map("${:,.2f}".format)
purchase_analysis_gender['Average Price'] = purchase_analysis_gender['Average Price'].map("${:,.2f}".format)
purchase_analysis_gender['Avg Total Purchase Per Person'] = purchase_analysis_gender['Avg Total Purchase Per Person'].map("${:,.2f}".format)
purchase_analysis_gender

,Number of Purchases,Total Revenue,Average Price,Avg Total Purchase Per Person
Gender,,,,
Female,113,$361.94,$3.11,$4.47
Male,652,"$1,967.64",$3.04,$4.07
Other / Non-Disclosed,15,$50.19,$3.31,$4.56


# Age Demographics

In [22]:
#select buckets for age grouping
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]
fantasy_df['Age Group'] = pd.cut(fantasy_df['Age'], bins, labels=labels)
players_df = fantasy_df.drop_duplicates(['SN'])
age_demo = pd.DataFrame(players_df['Age Group'].value_counts())
age_demo = age_demo.rename(columns={'Age Group': 'Total Count'})
age_demo['Percentage of Players (%)'] = round((age_demo['Total Count']/num_of_players)*100,2)
age_demo.sort_index()

,Total Count,Percentage of Players (%)
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


# Purchasing Analysis (Age)

In [23]:
#create metrics
age_summary = fantasy_df.groupby('Age Group').agg({'Item ID': 'count','Price': ['sum', lambda x: round(x.unique().mean(),2)]})
age_summary.columns = age_summary.columns.droplevel(level=1)
age_summary['Price'].fillna(0)

#average purchase by person by age group
age_summary2 = fantasy_df.groupby(['Age Group', 'SN']).agg({'Item ID': 'count', 'Price': 'sum'})
age_summary2 = age_summary2.dropna()

age_summary2 = age_summary2.groupby('Age Group').agg(Avg_Purchase_Tot = pd.NamedAgg(column = 'Price', aggfunc= lambda x: round(mean(x),2)))

#final summary
purchase_analysis_age = age_summary.join(age_summary2)
purchase_analysis_age['Avg_Purchase_Tot'] = purchase_analysis_age['Avg_Purchase_Tot'].fillna(0)

#rename columns
purchase_analysis_age.columns = ['Number of Purchases', 'Total Revenue', 'Average Price', 'Avg Total Purchase Per Person']

purchase_analysis_age['Average Price'] = purchase_analysis_age['Average Price'].fillna(0)
#format columns
purchase_analysis_age['Total Revenue'] = purchase_analysis_age['Total Revenue'].map("${:,.2f}".format)
purchase_analysis_age['Average Price'] = purchase_analysis_age['Average Price'].map("${:,.2f}".format)
purchase_analysis_age['Avg Total Purchase Per Person'] = purchase_analysis_age['Avg Total Purchase Per Person'].map("${:,.2f}".format)

purchase_analysis_age

,Number of Purchases,Total Revenue,Average Price,Avg Total Purchase Per Person
Age Group,,,,
<10,23,$77.13,$3.43,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$2.96,$3.86
20-24,365,"$1,114.06",$3.03,$4.32
25-29,101,$293.00,$2.97,$3.81
30-34,73,$214.00,$3.05,$4.12
35-39,41,$147.67,$3.53,$4.76
40+,13,$38.24,$2.94,$3.19


# Top Spenders

In [24]:
grouped_by_players = fantasy_df.groupby('SN').agg({'Purchase ID': 'count', 'Price': ['sum', lambda x: round(x.unique().mean(),2)]})
grouped_by_players.columns = grouped_by_players.columns.droplevel(level=1)

#rename columns
grouped_by_players.columns = ['Number of Purchases', 'Total Revenue', 'Average Price']

players_sort_by_revenue = grouped_by_players.sort_values(by=['Total Revenue'], ascending=False)

#format columns
players_sort_by_revenue['Total Revenue'] = players_sort_by_revenue['Total Revenue'].map("${:,.2f}".format)
players_sort_by_revenue['Average Price'] = players_sort_by_revenue['Average Price'].map("${:,.2f}".format)

top_five_players = players_sort_by_revenue.iloc[0:5]
top_five_players

,Number of Purchases,Total Revenue,Average Price
SN,,,
Lisosia93,5,$18.96,$3.79
Idastidru52,4,$15.45,$3.86
Chamjask73,3,$13.83,$4.61
Iral74,4,$13.62,$3.40
Iskadarya95,3,$13.10,$4.37


# Most Popular Items

In [25]:
grouped_by_items = fantasy_df.groupby(['Item ID', 'Item Name']).agg({'Purchase ID': 'count', 'Price': 'sum'})

#rename columns
grouped_by_items.columns = ['Number of Purchases', 'Total Revenue']

grouped_by_items = grouped_by_items.join(inventory_df)

#rename columns
grouped_by_items.columns = ['Number of Purchases', 'Total Revenue', 'Item Price']

#sort items by purchase count
items_sort_by_count= grouped_by_items.sort_values(by=['Number of Purchases'], ascending=False)

#format columns
items_sort_by_count['Total Revenue'] = items_sort_by_count['Total Revenue'].map("${:,.2f}".format)
items_sort_by_count['Item Price'] = items_sort_by_count['Item Price'].map("${:,.2f}".format)


top_five_items_count = items_sort_by_count.iloc[0:5]
top_five_items_count


#items_sort_by_count.head(15)
#ask instructor how to do rankings with equal spots


,,Number of Purchases,Total Revenue,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
145,Fiery Glass Crusader,9,$41.22,$4.58
108,"Extraction, Quickblade Of Trembling Hands",9,$31.77,$3.53
82,Nirvana,9,$44.10,$4.90
19,"Pursuit, Cudgel of Necromancy",8,$8.16,$1.02


# Most Profitable Items

In [26]:
#sort items by purchase count
items_sort_by_revenue= grouped_by_items.sort_values(by=['Total Revenue'], ascending=False)

#format columns
items_sort_by_revenue['Total Revenue'] = items_sort_by_revenue['Total Revenue'].map("${:,.2f}".format)
items_sort_by_revenue['Item Price'] = items_sort_by_revenue['Item Price'].map("${:,.2f}".format)


top_five_items_revenue = items_sort_by_revenue.iloc[0:5]
top_five_items_revenue

,,Number of Purchases,Total Revenue,Item Price
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$50.76,$4.23
82,Nirvana,9,$44.10,$4.90
145,Fiery Glass Crusader,9,$41.22,$4.58
92,Final Critic,8,$39.04,$4.88
103,Singed Scalpel,8,$34.80,$4.35
